## Mixing audios

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io.wavfile
from IPython.display import Audio
%matplotlib inline
import os

import pydub
from pydub import AudioSegment

In [ ]:
!pwd

In [ ]:
audio_names = ['Back_Vox', 'Bass', 'Drums', 'Guitars', 'Keys', 'Lead_Vox']

In [ ]:
for i in range(len(audio_names)):
    song = AudioSegment.from_mp3("band-music/" + audio_names[i] + ".mp3")
    song.export("band-music/" + audio_names[i] + ".wav", format="wav")

### Load raw data and preprocess

In [ ]:
audios = []
audio_lengths = []
for i in range(len(audio_names)):
    # Load audio (freq, data)
    f, s = scipy.io.wavfile.read('band-music/'+audio_names[i]+'.wav')
    
    # Pick only first channel for simplicity
    s = s[:,0]
    
    audios.append(s)
    audio_lengths.append(len(s))

In [ ]:
audios

In [ ]:
for audio in audios:
    print(audio.shape)

In [ ]:
# Cut audios to have same length
min_length = min(audio_lengths)

cropped_audios = []
for audio in audios:
    audio = audio[:min_length]
    cropped_audios.append(audio)

In [ ]:
min_length

### Visualize, play and save preprocessed audios

In [ ]:
i = 0
for audio in cropped_audios:
    
    # Plot waveforms
    plt.figure()
    plt.plot(audio)
    plt.xlabel('Sample Index')
    plt.ylabel('Amplitude')
    plt.title('Waveform of '+ audio_names[i])
    plt.show()
    
    # Play audios
    aud = Audio(audio, rate=f)
    display(aud)
    
    if not os.path.exists('band-music/sources/'):
            os.makedirs('band-music/sources/')

    # Save cropped audio sources to wav filecd s
    scipy.io.wavfile.write('band-music/sources/'+ audio_names[i] + '.wav', f, audio.astype(np.int16))
    i+=1

### Mix signals together

This section contains a function for mixing n audios given as input together and returning n mixed audio files that can be used for the ICA separation.

In [ ]:
def mix_audios(audios, save_path):
    # audios :: list of pre processed audios that have the same length
    # save_path :: path to save the mixed audios
    
    n_components = len(audios)
    
    # Concatenate audios
    S = audios[0]
    for i in range(1, len(audios)):
        S = np.c_[S, audios[i]]
    
    # Standardize and traspose data
    #S = S / S.std(axis=0)
    S = S.T
    
    # Create a mixing matrix
    A = np.eye(n_components)
    A = np.where(A==0, 0.5, A)
    
    # Mix signals
    X = np.dot(A,S)
    
    # Visualize, play and save mixed audios
    for i in range(X.shape[0]):
        
        s = X[i]
        
        # plot waveform
        plt.figure()
        plt.plot(s)
        plt.xlabel('Sample Index')
        plt.ylabel('Amplitude')
        plt.title('Mixed Signal %d ' % (i+1))
        plt.show()

        # Play audio
        aud = Audio(s, rate=f)
        display(aud)
        
        if not os.path.exists(save_path+'/'+str(n_components)+'components'):
            os.makedirs(save_path+'/'+str(n_components)+'components')

        # Save mixed sounds to wav file
        scipy.io.wavfile.write(save_path+'/'+str(n_components)+'components/mixed_' + str(i+1) + '.wav', f, s.astype(np.int16))


### Mix 2 audios together using above function

In [ ]:
to_mix = [cropped_audios[0], cropped_audios[1]]

In [ ]:
mix_audios(to_mix, 'band-music/mixed_audios')

### Mix 3 audios together using above function

In [ ]:
to_mix = [cropped_audios[0], cropped_audios[1], cropped_audios[2]]
mix_audios(to_mix, 'band-music/mixed_audios')

### Mix 4 audios together using above function

In [ ]:
to_mix = [cropped_audios[0], cropped_audios[1], cropped_audios[2], cropped_audios[3]]
mix_audios(to_mix, 'band-music/mixed_audios')

### Mix 5 audios together using above function

In [ ]:
to_mix = [cropped_audios[0], cropped_audios[1], cropped_audios[2], cropped_audios[3], cropped_audios[4]]
mix_audios(to_mix, 'band-music/mixed_audios')